In [2]:
!pip install -U torch torchvision
!pip install git+https://github.com/facebookresearch/fvcore.git
!git clone https://github.com/facebookresearch/detectron2 detectron2_repo
!pip install -e detectron2_repo

   ━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━ 306.0/766.7 MB 75.0 MB/s eta 0:00:07
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.1/150.1 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 77.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
import torch, torchvision

import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

import numpy as np
import cv2
from google.colab.patches import cv2_imshow

from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/MyDrive/dataset_CV_bootcamp/datasets
%ls

Mounted at /gdrive
/gdrive/MyDrive/dataset_CV_bootcamp/datasets
data.yaml  test/  train/


In [2]:
import detectron2
from detectron2.engine import DefaultPredictor, DefaultTrainer
from detectron2.config import get_cfg
from detectron2.model_zoo import get_config_file, get_checkpoint_url
from detectron2.data.datasets import register_coco_instances
from detectron2.utils.visualizer import Visualizer
from detectron2.engine import DefaultTrainer
import cv2
from google.colab.patches import cv2_imshow

register_coco_instances("my_train", {}, "/content/drive/MyDrive/dataset_CV_bootcamp/datasets/train/annotations.json", "/content/drive/MyDrive/dataset_CV_bootcamp/datasets/train/images")

cfg = get_cfg()
cfg.merge_from_file(get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("my_train",)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = get_checkpoint_url("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml") 
cfg.SOLVER.IMS_PER_BATCH = 2 
cfg.SOLVER.BASE_LR = 0.00025 
cfg.SOLVER.MAX_ITER = 3000 
cfg.SOLVER.STEPS = [] 
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128 
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3 
cfg.OUTPUT_DIR = "/gdrive/MyDrive/dataset_CV_bootcamp/output_1"

cfg.TEST.EVALUATOR = "COCOEvaluator"

trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

[03/13 12:56:49 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

model_final_f6e8b1.pkl: 243MB [00:02, 84.3MB/s]                          
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}


[03/13 12:56:54 d2.engine.train_loop]: Starting training from iteration 0


/usr/local/lib/python3.11/dist-packages/torch/functional.py:534: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3595.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[03/13 12:57:23 d2.utils.events]:  eta: 0:35:17  iter: 19  total_loss: 2.193  loss_cls: 1.43  loss_box_reg: 0.5164  loss_rpn_cls: 0.1334  loss_rpn_loc: 0.02743    time: 0.8894  last_time: 0.7082  data_time: 0.7192  last_data_time: 0.0014   lr: 4.9953e-06  max_mem: 3506M
[03/13 12:57:54 d2.utils.events]:  eta: 0:41:36  iter: 39  total_loss: 1.999  loss_cls: 1.321  loss_box_reg: 0.5679  loss_rpn_cls: 0.05436  loss_rpn_loc: 0.01702    time: 1.0347  last_time: 1.2042  data_time: 0.6288  last_data_time: 0.4910   lr: 9.9902e-06  max_mem: 3506M
[03/13 12:58:16 d2.utils.events]:  eta: 0:43:48  iter: 59  total_loss: 1.785  loss_cls: 1.167  loss_box_reg: 0.5038  loss_rpn_cls: 0.0428  loss_rpn_loc: 0.02211    time: 1.0647  last_time: 0.9314  data_time: 0.5326  last_data_time: 0.1397   lr: 1.4985e-05  max_mem: 3545M
[03/13 12:58:35 d2.utils.events]:  eta: 0:42:53  iter: 79  total_loss: 1.756  loss_cls: 0.9802  loss_box_reg: 0.6741  loss_rpn_cls: 0.0449  loss_rpn_loc: 0.01976    time: 1.0345  last_

FileNotFoundError: [Errno 2] No such file or directory: 'datasets/coco/annotations/instances_val2017.json'

In [7]:
import glob
import os

checkpoints = sorted(glob.glob(os.path.join("/gdrive/MyDrive/dataset_CV_bootcamp/output_1", "model_*.pth")))

if checkpoints:
    weight_path = checkpoints[-1]
    print(f"Последний найденный чекпоинт: {weight_path}")


    cfg.MODEL.WEIGHTS = weight_path
    predictor = DefaultPredictor(cfg)
    print("Весы успешно загружены!")
else:
    print("Чекпоинты не найдены.")

Последний найденный чекпоинт: /gdrive/MyDrive/dataset_CV_bootcamp/output_1/model_final.pth
[03/13 13:54:00 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /gdrive/MyDrive/dataset_CV_bootcamp/output_1/model_final.pth ...


/usr/local/lib/python3.11/dist-packages/fvcore/common/checkpoint.py:252: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(f, map_location=torch.device("cpu"))

Весы успешно загружены!


In [12]:
import shutil

destination = "/content/drive/MyDrive/dataset_CV_bootcamp/datasets/"
print(f'Exist:{os.path.exists(weight_path)}')

shutil.copy(os.path.join("/gdrive/MyDrive/dataset_CV_bootcamp/output_1", weight_path), destination)
print(f"Файл с весами скопирован в папку: {destination}")


Exist:False
Файл с весами скопирован в папку: /content/drive/MyDrive/dataset_CV_bootcamp/datasets/


In [13]:
from detectron2.data.datasets import register_coco_instances
from detectron2.evaluation import COCOEvaluator
from detectron2.engine import DefaultPredictor
from detectron2.evaluation import inference_on_dataset
from detectron2.data import build_detection_test_loader

from detectron2.config import get_cfg

register_coco_instances(
    "my_val2",
    {},
    "/content/drive/MyDrive/dataset_CV_bootcamp/datasets/train/annotations_val.json",
    "/content/drive/MyDrive/dataset_CV_bootcamp/datasets/train/images"
)

cfg = get_cfg()
cfg.merge_from_file(get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"))
cfg.DATASETS.TEST = ("my_val2",) 
cfg.MODEL.WEIGHTS = "/gdrive/MyDrive/dataset_CV_bootcamp/output_1/model_final.pth" 
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5 

class CustomCOCOEvaluator(COCOEvaluator):
    def __init__(self, dataset_name, output_dir=None):
        super().__init__(dataset_name, output_dir=output_dir)
        self._iou_thresholds = [0.5] 


predictor_val = DefaultPredictor(cfg)

val_loader = build_detection_test_loader(cfg, "my_val2")

evaluator = CustomCOCOEvaluator("my_val2", output_dir=cfg.OUTPUT_DIR)

results = inference_on_dataset(predictor_val.model, val_loader, evaluator)
print(results)


[03/13 14:06:56 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /gdrive/MyDrive/dataset_CV_bootcamp/output_1/model_final.pth ...


/usr/local/lib/python3.11/dist-packages/fvcore/common/checkpoint.py:252: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(f, map_location=torch.device("cpu"))

WARNING [03/13 14:06:58 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[03/13 14:06:58 d2.data.datasets.coco]: Loaded 371 images in COCO format from /content/drive/MyDrive/dataset_CV_bootcamp/datasets/train/annotations_val.json
[03/13 14:06:58 d2.data.build]: Distribution of instances among all 3 categories:
|  category  | #instances   |  category  | #instances   |  category  | #instances   |
|:----------:|:-------------|:----------:|:-------------|:----------:|:-------------|
|   plane    | 408          |    car     | 1240         |    ship    | 193          |
|            |              |            |              |            |              |
|   total    | 1841         |            |              |            |              |
[03/13 14:06:58 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[03/13 14:06:5

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


[03/13 14:07:02 d2.evaluation.evaluator]: Inference done 11/371. Dataloading: 0.1105 s/iter. Inference: 0.1357 s/iter. Eval: 0.0001 s/iter. Total: 0.2464 s/iter. ETA=0:01:28
[03/13 14:07:07 d2.evaluation.evaluator]: Inference done 31/371. Dataloading: 0.1179 s/iter. Inference: 0.1353 s/iter. Eval: 0.0001 s/iter. Total: 0.2536 s/iter. ETA=0:01:26
[03/13 14:07:12 d2.evaluation.evaluator]: Inference done 56/371. Dataloading: 0.1001 s/iter. Inference: 0.1349 s/iter. Eval: 0.0001 s/iter. Total: 0.2353 s/iter. ETA=0:01:14
[03/13 14:07:17 d2.evaluation.evaluator]: Inference done 79/371. Dataloading: 0.0936 s/iter. Inference: 0.1369 s/iter. Eval: 0.0001 s/iter. Total: 0.2307 s/iter. ETA=0:01:07
[03/13 14:07:23 d2.evaluation.evaluator]: Inference done 100/371. Dataloading: 0.0979 s/iter. Inference: 0.1360 s/iter. Eval: 0.0001 s/iter. Total: 0.2341 s/iter. ETA=0:01:03
[03/13 14:07:28 d2.evaluation.evaluator]: Inference done 123/371. Dataloading: 0.0957 s/iter. Inference: 0.1354 s/iter. Eval: 0.0

In [14]:
def get_label_from_COCO(instances, image_width, image_height):
    labels = []
    for i in range(len(instances)):
        class_id = instances.pred_classes[i].item()
        bbox = instances.pred_boxes[i].tensor.cpu().numpy()[0]

    
        x_center_norm = (bbox[0] + bbox[2]) / 2 / image_width
        y_center_norm = (bbox[1] + bbox[3]) / 2 / image_height
        width_norm = (bbox[2] - bbox[0]) / image_width
        height_norm = (bbox[3] - bbox[1]) / image_height

    
        label = f"{class_id} {x_center_norm:.4f} {y_center_norm:.4f} {width_norm:.4f} {height_norm:.4f}"
        labels.append(label) 


    tab_separated = "\t".join(labels) 
    return f'"[{tab_separated}]"' if labels else '"[]"'


In [15]:
import pandas as pd
import os
import cv2
from tqdm import tqdm
import csv

test_dir = '/gdrive/MyDrive/dataset_CV_bootcamp/datasets/test/images'

image_ids = [f for f in os.listdir(test_dir) if f.endswith('.jpg')]

predictions = []

for image_id in tqdm(image_ids):
    image_path = os.path.join(test_dir, image_id)
    image = cv2.imread(image_path)
    h, w = image.shape[:2]

    results = predictor_val(image)
    instances = results['instances'].to('cpu')


    formatted_label = get_label_from_COCO(instances, w, h)

    predictions.append([image_id, formatted_label])

submission_df = pd.DataFrame(predictions, columns=['id', 'label'])

submission_df.to_csv(
    os.path.join(cfg.out),
    index=False,
    quoting=csv.QUOTE_NONE, 
    escapechar='\\' 
)
print()
print('Предсказания сохранены в submission.csv')


FileNotFoundError: [Errno 2] No such file or directory: '/gdrive/MyDrive/dataset_CV_bootcamp/datasets/test/images'